In [1]:
import urllib
from bs4 import BeautifulSoup as soup
import pandas as pd

In [3]:
#link for one category
url = "https://cookieandkate.com/category/food-recipes/breakfast/"

In [4]:
#using browser to scrape
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

In [6]:
#getting data from page
request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read()
response.close()

page = soup(data,"html.parser")

In [27]:
#list of all recipe links
recipepages = []
nextpage = page

In [28]:
while True:

    #next page marker
    nextmarker = nextpage.find("div", {"class":"pagination-next alignright"})
    
    #if there is a next page
    if nextmarker:
        
        #get url for page
        nexturl = nextmarker.a['href']
        
        #read info from page
        nextrequest = urllib.request.Request(nexturl,None,headers)
        nextresponse = urllib.request.urlopen(nextrequest)
        nextdata = nextresponse.read()
        nextresponse.close()

        nextpage = soup(nextdata)
        
        #collect all recipes
        recipes = nextpage.findAll("div",{"class":"lcp_catlist_item"})
        
        #save their links
        for recipe in recipes:
            recipepages.append(recipe.a["href"])
    
    #if there are no new pages
    else:
        break

https://cookieandkate.com/category/food-recipes/breakfast/page/2/
https://cookieandkate.com/category/food-recipes/breakfast/page/3/
https://cookieandkate.com/category/food-recipes/breakfast/page/4/
https://cookieandkate.com/category/food-recipes/breakfast/page/5/
https://cookieandkate.com/category/food-recipes/breakfast/page/6/
https://cookieandkate.com/category/food-recipes/breakfast/page/7/


In [34]:
recipeinfo = []
for newurl in recipepages:
    newdict = {}
    #reading info from recipe page
    newrequest = urllib.request.Request(newurl,None,headers) #The assembled request
    newresponse = urllib.request.urlopen(newrequest)
    newdata = newresponse.read()
    newresponse.close()
    
    recipepage = soup(newdata,"html.parser")
    
    recipedetails = recipepage.find("div",{"class":"tasty-recipes-details"})
    reciperatings = recipepage.find("div",{"class":"tasty-recipes-ratings"})
    
    if recipedetails and reciperatings:
        newdict["title"] = recipepage.h1.text

        #recipe details

        newdict["time"] = recipedetails.find("span",{"class":"tasty-recipes-total-time"}).text
        try:
            newdict["cuisine"] = recipedetails.find("span",{"class":"tasty-recipes-cuisine"}).text
        
            newdict["category"] = recipedetails.find("span",{"class":"tasty-recipes-category"}).text
        except:
            newdict["cuisine"] = None
            newdict["category"] = None

        #recipe ratings
        newdict["rating"] = reciperatings.find("span",{"class":"average"}).text

        #adding dictionary of recipe info to list
        recipeinfo.append(newdict)

In [35]:
pd.DataFrame(recipeinfo)

,title,time,cuisine,category,rating
0,Strawberry Oat Muffins,40 minutes,American,Muffins,4.7
1,Simple Breakfast Quesadillas,25 minutes,Mexican,Entree,5
2,Homemade Bloody Marys,10 minutes,American,Cocktail,4.4
3,How to Make Frittatas (Stovetop or Baked),45 minutes,Italian,Breakfast,5
4,Basic Blueberry Smoothie,5 minutes,American,Smoothie,4.9
...,...,...,...,...,...
102,Honey Whole Wheat Banana Bread,1 hour 5 minutes,None,None,4.8
103,Breakfast Quinoa,5 mins,None,None,5
104,Blueberry Lemon Muffins,35 minutes,None,None,4.9
105,Coconut Pancakes,25 minutes,None,None,4.9
